This notebook showcases the yfinance timeseries feature as well as the BollingerBand anomaly detection features in news-signals library added in v0.8.0.

Lets get necessary non news-signals imports for this notebook

In [ ]:
import copy
import requests
import json
import datetime
import matplotlib.pyplot as plt

Lets import the needed news-signals features. 

Put your NewsAPI Key and ID in **NEWSAPI_APP_KEY** and **NEWSAPI_APP_ID**. 

Although it is not necessary and you can comment out building a pre-existing news-volume signal and just get the finance timeseries further down in the notebook.

In [ ]:
from news_signals import signals
NEWSAPI_APP_KEY= ""
NEWSAPI_APP_ID= ""
from news_signals import signals, newsapi, wikidata_utils
newsapi.set_headers(NEWSAPI_APP_ID, NEWSAPI_APP_KEY)
from news_signals.anomaly_detection import SigmaAnomalyDetector, BollingerAnomalyDetector

In [ ]:
# utility function for getting a wikidata id from an entity name

WIKIDATA_SEARCH_URL = "https://www.wikidata.org/w/api.php"
DEFAULT_SEARCH_PARAMS = {
    "action": "wbsearchentities",
    "format": "json",
    "errorformat": "plaintext",
    "language": "en",
    "uselang": "en",
    "type": "item",
    "limit": 1
}
WD_ENTITY_BASE_URL = 'https://www.wikidata.org/wiki/Special:EntityData'
    

def search_wikidata(surface_form, min_length=3):
    params = copy.deepcopy(DEFAULT_SEARCH_PARAMS)
    params["search"] = surface_form
    result = []
    try:
        print(f'querying wikidata with params: {params}')
        r = requests.get(url=WIKIDATA_SEARCH_URL, params=params)
        data = json.loads(r.text)
        if 'search' in data:
            result = data['search']
            result = result[0:min(100, len(result))]
    except Exception as e:
        print(f'Error searching wikidata for surface form: {surface_form}')
        print(e)
    return result

This is setting up to get the news volume signals. 

Feel free to modify **entity_name** !

In [ ]:
# let's setup the entity we want to work with

entity_name = 'Twitter'

entity_id_candidates = search_wikidata(entity_name)
test_entity = entity_id_candidates[0]

Here we instantiate the AylienSignal class. 

If you do not have an NewsAPI key, you can blank instantiate the class and just add the yfinance timeseries later on.

In [ ]:

signal = signals.AylienSignal(
    name=test_entity['label'],
    params={"entity_ids": [test_entity['id']]}
)

Here we define the start to end dates for the news volume timeseries and get the signal. Feel free to modify this too !

In [ ]:
start = '2023-01-01'
end = '2024-02-21'

timeseries_signal = signal(start, end)

Now we can plot our news volume timeseries

In [ ]:
signal.plot()

Now we can finally add our yfinance market timeseries.

About the function:

The date range is determined from self.start and self.end (if available),
or from the minimum and maximum dates of self.timeseries_df's index.

Parameters:
- **ticker (str)**: The stock ticker symbol to retrieve (required).
- **columns (str or list of str, optional)**: The column(s) to extract from the yfinance data.
    Defaults to ["Close"] if rsi is False, or ["Close", "RSI"] if rsi is True.
- **rsi (bool)**: Whether to request RSI data. The underlying retrieval function uses this flag.
- **overwrite_existing (bool)**: Whether to overwrite existing yfinance data if already present.
- **append_dates (bool)**:
    If True, any dates present in the yfinance data that are not already in self.timeseries_df
    will be appended (the DataFrame is reindexed to the union of dates).
    If False, only rows with dates common to both DataFrames are updated.

Returns:
- self: The signal instance with the new timeseries data.

In [ ]:
timeseries_signal.add_yfinance_market_timeseries(
    ticker="AAPL",
    columns=["Open","Close","RSI"],
    rsi=True,
)

print(timeseries_signal.timeseries_df)

Now finally, lets do Bollinger Band Anomaly Detection on our retrived yfinance data

In [ ]:
# Drop unnecessary columns
filtered_df = timeseries_signal.timeseries_df.drop(columns=["count", "published_at"])

# Drop rows where 'open' and 'close' are NaN
filtered_df = filtered_df.dropna(subset=["open", "close"])

# Initialize Bollinger anomaly detector
bollinger_detector = BollingerAnomalyDetector(window=20, num_std=2.0)

# Apply Bollinger Anomaly Detection on 'close' prices
bollinger_anomalies = bollinger_detector(history=filtered_df["close"], series=filtered_df["close"])

# Create a single plot for Bollinger anomalies
plt.figure(figsize=(12, 6))

# Plot closing prices
plt.plot(filtered_df.index, filtered_df["close"], label="Closing Price", color="blue")

# Identify and plot anomalies
bollinger_anomalies = bollinger_anomalies.reindex(filtered_df.index)
anomaly_points = filtered_df["close"][bollinger_anomalies > 0]

plt.scatter(anomaly_points.index, anomaly_points, color="green", label="Bollinger Anomalies", marker="o")

# Labels and legend
plt.title("Bollinger Band Anomaly Detection")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()

plt.show()
